In [6]:
import dlib
import cv2
import os
import re
import json

Repo: https://github.com/xinyooo/deepfake-detection

Dataset: https://www.kaggle.com/c/deepfake-detection-challenge/data

In [7]:
train_frame_folder = 'train_sample_videos'
with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)
list_of_train_data = [f for f in os.listdir(
    train_frame_folder) if f.endswith('.mp4')]
detector = dlib.get_frontal_face_detector()
for vid in list_of_train_data:
    count = 0
    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))
    frameRate = cap.get(5)
    while cap.isOpened():
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()

                crop_img = frame[y1:y2, x1:x2]
                h, w, channels = crop_img.shape
                half = h//2
                top = crop_img[:half, :]
                bottom = crop_img[half:, :]

                if data[vid]['label'] == 'REAL':
                    cv2.imwrite('dataset/real/'+vid.split('.')
                                [0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                    cv2.imwrite('dataset/real/'+vid.split('.')
                                [0]+'_'+str(count)+'top'+'.png', cv2.resize(top, (128, 128)))
                    cv2.imwrite('dataset/real/'+vid.split('.')
                                [0]+'_'+str(count)+'bottom'+'.png', cv2.resize(bottom, (128, 128)))
                elif data[vid]['label'] == 'FAKE':
                    cv2.imwrite('dataset/fake/'+vid.split('.')
                                [0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                    cv2.imwrite('dataset/fake/'+vid.split('.')
                                [0]+'_'+str(count)+'top'+'.png', cv2.resize(top, (128, 128)))
                    cv2.imwrite('dataset/fake/'+vid.split('.')
                                [0]+'_'+str(count)+'bottom'+'.png', cv2.resize(bottom, (128, 128)))
                count += 1
